In [62]:
# 把之前的一些操作刪掉
# 刪掉所有Namespaces
! kubectl delete namespaces --all
# 刪掉所有資源
! kubectl delete all --all 
# 把預設 namespace 恢復回 default
! kubectl config set-context --current --namespace=default

namespace "kube-node-lease" deleted
Error from server (Forbidden): namespaces "default" is forbidden: this namespace may not be deleted
Error from server (Forbidden): namespaces "kube-public" is forbidden: this namespace may not be deleted
Error from server (Forbidden): namespaces "kube-system" is forbidden: this namespace may not be deleted
pod "hostname-695cf7968c-rlzrp" deleted
service "kubernetes" deleted
deployment.apps "hostname" deleted
replicaset.apps "hostname-695cf7968c" deleted
Context "minikube" modified.


# Namespaces 

- Namespaces 提供一個方式把資源隔離
- kubectl 的默認 namespaces 是 `default`
- 用法：同一個應用使用共同Namespaces


In [157]:
# 查看目前使用的預設Namesapce名稱
! kubectl config get-contexts 

CURRENT   NAME       CLUSTER    AUTHINFO   NAMESPACE
*         minikube   minikube   minikube   development


In [80]:
# 列出目前有的所有Namespaces 
! kubectl get namespaces

NAME              STATUS   AGE
default           Active   31s
kube-node-lease   Active   33s
kube-public       Active   33s
kube-system       Active   33s


## initial namespaces

minikube(kubernetes)有4個內建的Namespaces

- `default`: 默認namespaces
- `kube-node-lease`: 處理lease租約。# 空的
- `kube-public`: 公開namespaces, 爲經認證的節點也可以存取。# 空的
- `kube-system`: created by kube-system, 包含維持kubernetes正常運行的deployment

In [81]:
# 列印目前所有資源 (default)
# 沒有資源是正常的，因爲你什麼資源都沒建立
! kubectl get all

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   35s


In [82]:
! kubectl get all -n kube-node-lease

No resources found in kube-node-lease namespace.


In [83]:
! kubectl get all -n kube-public

No resources found in kube-public namespace.


In [85]:
# 列印在 kube-system namespaces 的所有資源
! kubectl get all -n kube-system

NAME                                   READY   STATUS    RESTARTS      AGE
pod/coredns-565d847f94-qw6qd           1/1     Running   0             42s
pod/etcd-minikube                      1/1     Running   0             56s
pod/kube-apiserver-minikube            1/1     Running   0             54s
pod/kube-controller-manager-minikube   1/1     Running   0             54s
pod/kube-proxy-2p6b8                   1/1     Running   0             42s
pod/kube-scheduler-minikube            1/1     Running   0             54s
pod/storage-provisioner                1/1     Running   1 (11s ago)   51s

NAME               TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)                  AGE
service/kube-dns   ClusterIP   10.96.0.10   <none>        53/UDP,53/TCP,9153/TCP   55s

NAME                        DESIRED   CURRENT   READY   UP-TO-DATE   AVAILABLE   NODE SELECTOR            AGE
daemonset.apps/kube-proxy   1         1         1       1            1           kubernetes.io/os=linux   55s

NAM

In [86]:
# 建立一個Deploy # 沒有制度Namespace
! kubectl create deployment hostname \
  --image=docker.io/brokenpen/hostname:v1

deployment.apps/hostname created


In [87]:
# 把 deployment.apps/hostname 加上 env=prod 這個標籤
! kubectl label \
  --overwrite deployment hostname env=prod

deployment.apps/hostname labeled


In [88]:
# 列印 deployment.apps/hostname的資訊
! kubectl get deploy hostname \
  -o custom-columns="Kind:.kind,Namespace:.metadata.namespace, Name:.metadata.name, Labels:.metadata.labels"

Kind         Namespace    Name       Labels
Deployment   default     hostname   map[app:hostname env:prod]


In [89]:
# 描述 deployment.apps/hostname
! kubectl describe deployment hostname 

Name:                   hostname
Namespace:              default
CreationTimestamp:      Wed, 30 Nov 2022 08:11:03 +0000
Labels:                 app=hostname
                        env=prod
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=hostname
Replicas:               1 desired | 1 updated | 1 total | 0 available | 1 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=hostname
  Containers:
   hostname:
    Image:        docker.io/brokenpen/hostname:v1
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      False   MinimumReplicasUnavailable
  Progressing    True    ReplicaSetUpdated
OldReplicaSets:  <none>
NewReplicaSet:   hostname-695cf7968c (1/1 replicas created)
Events:

In [91]:
# 建立資源時指定 
# namespaces: kube-public
! kubectl create deployment hostname \
  --image=docker.io/brokenpen/hostname:v1 \
  --namespace=kube-public

error: failed to create deployment: deployments.apps "hostname" already exists


In [92]:
# 列印目前所有 hostname deployment
# 包含所有 namespaces deployment
! kubectl get deploy \
  --all-namespaces \
  --selector=app=hostname

NAMESPACE     NAME       READY   UP-TO-DATE   AVAILABLE   AGE
default       hostname   0/1     1            0           46s
kube-public   hostname   0/1     1            0           35s


---

---


#### 目前還只是使用 minikube 的預設 namespace

#### 嘗試下自己建立一個新的 namespaces 

In [96]:
# 建立一個 namesapces 爲 developemnt 的 namespaces 
! kubectl create namespace development

Error from server (AlreadyExists): namespaces "development" already exists


In [97]:
# 幫剛出建立的 namesapces
# 增加一個 labels 
! kubectl label \
  --overwrite namespace development env=dev

namespace/development not labeled


In [98]:
# 取得 namesapce development 的 yaml
! kubectl get namespaces development -o yaml

apiVersion: v1
kind: Namespace
metadata:
  creationTimestamp: "2022-11-30T08:11:59Z"
  labels:
    env: dev
    kubernetes.io/metadata.name: development
  name: development
  resourceVersion: "498"
  uid: 9082a610-a591-46cc-91c9-a2d2131e4680
spec:
  finalizers:
  - kubernetes
status:
  phase: Active


---
- namespace.yaml
  - apiVersion: v1
  - kind: Namespaces
  - metadata:
     - name: development
---

In [100]:
# 把 目前 kubectl 的預設namesapce 改爲 development
! kubectl config set-context \
  --current --namespace="development"

Context "minikube" modified.


In [101]:
# 列印目前的 config contexts
# 想要知道目前的Namesapces是否已改爲 development
! kubectl config get-contexts

CURRENT   NAME       CLUSTER    AUTHINFO   NAMESPACE
*         minikube   minikube   minikube   development


In [102]:
# 在新的 namespaces 建立 新的 deployment
! kubectl create deployment hostname\
  --image=docker.io/brokenpen/hostname:v1

deployment.apps/hostname created


In [111]:
# 幫剛出建立的 namesapces
# 增加一個 labels 
! kubectl label \
  --overwrite deployment hostname env=dev

deployment.apps/hostname labeled


In [112]:
# 列印目前(namesapces)的 deployment
! kubectl get deployment --show-labels

NAME       READY   UP-TO-DATE   AVAILABLE   AGE    LABELS
hostname   1/1     1            1           3m3s   app=hostname,env=dev


In [113]:
# 列印目前 development namesapces的 deployment
! kubectl get deployment -n development --show-labels

NAME       READY   UP-TO-DATE   AVAILABLE   AGE    LABELS
hostname   1/1     1            1           3m6s   app=hostname,env=dev


---
- 在使用 `kubectl get <reousrce-name>`
- 可以加上 `--all-namespaces` 顯示所有 namespaces 的對於 resource 的yaml
- `--selector` 是透過label比對進行選擇

In [129]:
# 列印在不同 namespaces 的 hostname deployment
! kubectl get deployment \
  --all-namespaces \
  --selector=app=hostname

NAMESPACE     NAME       READY   UP-TO-DATE   AVAILABLE   AGE
default       hostname   1/1     1            1           18m
development   hostname   1/1     1            1           14m
kube-public   hostname   1/1     1            1           18m


In [131]:
# 列印在不同 namespaces 的 hostname deployment
# 加上 labels 一同先顯示
! kubectl get deployment\
  --all-namespaces\
  --selector=app=hostname\
  -o custom-columns="NAMESPCE:.metadata.namespace,NAME:.metadata.name,LABELS:.metadata.labels"

NAMESPCE      NAME       LABELS
default       hostname   map[app:hostname env:prod]
development   hostname   map[app:hostname env:dev]
kube-public   hostname   map[app:hostname]


In [132]:
# 查看default namespaces 的 deployment
! kubectl get deployment --namespace=default

NAME       READY   UP-TO-DATE   AVAILABLE   AGE
hostname   1/1     1            1           18m


In [128]:
# 查看 development namespaces 的 pods
! kubectl get pods -n development

NAME                        READY   STATUS    RESTARTS   AGE
hostname-695cf7968c-f5kdv   1/1     Running   0          9m48s


In [140]:
! kubectl get pods \
  --all-namespaces \
  --selector=app=hostname 

NAMESPACE     NAME                        READY   STATUS    RESTARTS   AGE
default       hostname-695cf7968c-fxb9q   1/1     Running   0          21m
development   hostname-695cf7968c-f5kdv   1/1     Running   0          17m
kube-public   hostname-695cf7968c-tjq2j   1/1     Running   0          21m


---
刪除namespaces會把這個namespaces底下的所有resource object一同刪掉

In [141]:
# 刪除 development這個namespace
! kubectl delete namespace development

namespace "development" deleted


In [142]:
# 查看在 development namespaces地下的資源
! kubectl get all -n development

No resources found in development namespace.


In [143]:
# 列印在不同 namespaces 的 hostname deployment
# 剩下兩個
! kubectl get deployment \
  --all-namespaces \
  --selector=app=hostname

NAMESPACE     NAME       READY   UP-TO-DATE   AVAILABLE   AGE
default       hostname   1/1     1            1           25m
kube-public   hostname   1/1     1            1           25m


In [144]:
# 刪除 development這個namespace
! kubectl delete namespace default

Error from server (Forbidden): namespaces "default" is forbidden: this namespace may not be deleted


In [145]:
# default, inital namespaces 不能刪除

In [155]:
# 所有你這能刪除 default namespace 底下的 reousrces 
! kubectl delete all -l app=hostname -n default

No resources found


In [161]:
# 刪除 development這個namespace
! kubectl delete namespace kube-public

Error from server (Forbidden): namespaces "kube-public" is forbidden: this namespace may not be deleted


In [162]:
# kube-public, inital namespaces 不能刪除

In [163]:
# 所有你只能刪除 kube-public namespace 底下的 reousrces 
! kubectl delete all -l app=hostname -n kube-public

No resources found


In [160]:
# 把預設 namespace 恢復回 default
! kubectl config set-context --current --namespace=default

Context "minikube" modified.
